In [11]:
import pandas as pd
import pandasql as pdsql

In [46]:
article_views = pd.read_csv("linkedin_data.csv")

In [47]:
article_views

,date,viewer_id,article_id,author_id
0,8/1/2017,123,456,789
1,8/1/2017,123,456,789
2,8/2/2017,432,543,654
3,8/1/2017,789,789,789
4,8/1/2017,123,789,789
5,8/3/2017,567,780,432
6,8/2/2017,789,789,789


In [48]:
article_views.dtypes

date          object
viewer_id      int64
article_id     int64
author_id      int64
dtype: object

In [49]:
pysql = lambda q: pdsql.sqldf(q, globals())

In [50]:
sql = """select count(distinct author_id)
            from article_views
            where viewer_id not in
            (select viewer_id from article_views where viewer_id = author_id)"""
df = pysql(sql)

In [51]:
df

,count(distinct author_id)
0,3


In [56]:
self_view = article_views[article_views.viewer_id == article_views.author_id][["viewer_id"]].drop_duplicates("viewer_id")
not_self_view = article_views.merge(self_view, left_on = "viewer_id", right_on = "viewer_id", how = "left", indicator = True)
print(not_self_view[not_self_view["_merge"]=="left_only"].drop_duplicates("author_id").shape[0])

3


In [57]:
#not_self_view

In [44]:
sql2 = """select count(viewer_id)
            from
            (select viewer_id
                from article_views
                where date  = "8/1/2017"
            group by viewer_id
            having count(distinct article_id) > 1)"""
df2 = pysql(sql2)

In [45]:
df2

,count(viewer_id)
0,1


In [59]:
article_views["date_format"] =pd.to_datetime(article_views["date"])

In [63]:
# articles viewed on 2017-08-01
article_0801 = article_views[article_views.date_format == '2017-08-01']
article_0801

,date,viewer_id,article_id,author_id,date_format
0,8/1/2017,123,456,789,2017-08-01
1,8/1/2017,123,456,789,2017-08-01
3,8/1/2017,789,789,789,2017-08-01
4,8/1/2017,123,789,789,2017-08-01


In [72]:
df_unq = article_0801.groupby("viewer_id")["article_id"].value_counts().to_frame().drop("article_id", axis = 1).reset_index().groupby("viewer_id").count()
df_unq_multi = df_unq[df_unq.article_id > 1]
df_unq_multi.shape[0]

1

In [79]:
df_unq2 = article_0801.groupby("viewer_id")["article_id"].nunique().to_frame()
df_unq2[df_unq2.article_id > 1].shape[0]

1